# Data Cleaning

In [1]:
import pandas as pd

In [2]:
# Import Data
df = pd.read_csv('glassdoor_jobs.csv')
df.shape

(958, 14)

## Salary Parsing

In [3]:
# Parse if salary provided is hourly or employer provided 
df['hourly'] = df['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)
df['employer provided'] = df['Salary Estimate'].apply(lambda x: 1 if 'employer provided salary:' in x.lower() else 0)
df.shape

(958, 16)

In [4]:
# Remove Jobs with no salary estimate
df = df [ df['Salary Estimate']  != '-1']
df.shape

(783, 16)

In [5]:
# Remove ""(Glassdoor est.)" text in Salary estimate column
salary = df['Salary Estimate'].apply(lambda x: x.split('(')[0] ).to_frame()
salary = salary [ salary['Salary Estimate']  != ''].iloc[:,0]

# Remove '$', and 'k' text in Salary estimate column
minus_k = salary.apply(lambda x: x.replace('K', '').replace('$',''))
min_hr = minus_k.apply(lambda x: x.lower().replace('per hour', '').replace('employer provided salary:', ''))

# Create column for min_salary and max_salary
df['min_salary'] = min_hr.apply(lambda x: int(x.split('-')[0]))
df['max_salary'] = min_hr.apply(lambda x: int(x.split('-')[1].replace(" ", "")))

# Convert hourly wage to annual
df['min_salary'] = df.apply(lambda x: x.min_salary*2.08 if x.hourly == 1 else x.min_salary, axis = 1)
df['max_salary'] = df.apply(lambda x: x.max_salary*2.08 if x.hourly == 1 else x.max_salary, axis = 1)

# Create column for ave_salary
df['avg_salary'] = (df.min_salary + df.max_salary) / 2.08

# Remove na salary
df = df.dropna(subset = ['min_salary'])

df[['min_salary','max_salary', 'avg_salary']]

,min_salary,max_salary,avg_salary
0,67.0,81.0,71.153846
1,47.0,73.0,57.692308
2,69.0,105.0,83.653846
3,92.0,105.0,94.711538
4,65.0,73.0,66.346154
...,...,...,...
951,82.0,106.0,90.384615
952,58.0,72.0,62.500000
953,121.0,162.0,136.057692
954,83.0,111.0,93.269231


## Company name, age, location, and competitor parsing

In [6]:
# Company Name text only
df['company_txt'] = df.apply(lambda x: x['Company Name'].replace('\n', '') if x['Rating'] < 0 else x['Company Name'][:-3].replace('\n', ''), axis = 1)
df[['company_txt']]

,company_txt
0,AK Steel
1,MiTek Industries
2,Advanced Conversion Technology
3,"IDEAL AEROSMITH, INC"
4,Franklin Energy
...,...
951,Cushman & Wakefield
952,Woolpert
953,pSemi
954,Cosmic AES


In [7]:
# Total Jobs in each state
df['job_state'] = df['Location'].apply(lambda x: x.split(',')[1])
df['job_state'] = df['job_state'].apply(lambda x: x.upper().replace('LOS ANGELES', 'LA'))
df.job_state.value_counts()

 CA    157
 MD     60
 VA     58
 MA     50
 MI     48
 NY     38
 TX     26
 CO     23
 CT     22
 OR     20
 AZ     19
 OH     19
 IL     18
 GA     16
 PA     15
 UT     15
 WA     14
 AL     13
 FL     12
 WI     12
 NC     12
 DC     12
 MN     11
 LA     11
 NJ     11
 IN     10
 MO      9
 KY      7
 TN      7
 SC      6
 MS      4
 IA      4
 AK      4
 RI      4
 NH      3
 DE      2
 OK      2
 WV      1
 NV      1
 MT      1
 KS      1
 VT      1
 NE      1
 ND      1
Name: job_state, dtype: int64

In [8]:
# If the jobs and company's headquater is in the same state 0 = No, 1 = Yes
df['hq_same_state'] = df.apply(lambda x: 1 if x.Location == x.Headquarters else 0, axis = 1)
df.hq_same_state.value_counts()

0    480
1    301
Name: hq_same_state, dtype: int64

In [9]:
# Age of the company
# age of company
df['company_age'] = df.Founded.apply(lambda x: x if x <1 else 2020 -x)
df[['company_age']]

,company_age
0,31
1,65
2,-1
3,-1
4,-1
...,...
951,103
952,109
953,30
954,19


In [10]:
# Competitor count
df['Num_Competitors'] = df['Competitors'].apply(lambda x: len(x.split(',')) if x != '-1' else 0)
df.Num_Competitors.value_counts()

0    463
3    205
2     87
1     23
4      3
Name: Num_Competitors, dtype: int64

## Parsing length & keywords in job description

In [11]:
# Python
df['python'] = df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)
df.python.value_counts()

0    670
1    111
Name: python, dtype: int64

In [12]:
# MATLAB
df['matlab'] = df['Job Description'].apply(lambda x: 1 if 'matlab' in x.lower() else 0)
df.matlab.value_counts()

0    709
1     72
Name: matlab, dtype: int64

In [13]:
# C++
df['cpp'] = df['Job Description'].apply(lambda x: 1 if 'c++' in x.lower() else 0)
df.cpp.value_counts()

0    687
1     94
Name: cpp, dtype: int64

In [14]:
# SQL
df['sql'] = df['Job Description'].apply(lambda x: 1 if 'sql' in x.lower() else 0)
df.sql.value_counts()

0    737
1     44
Name: sql, dtype: int64

In [15]:
# Java
df['java'] = df['Job Description'].apply(lambda x: 1 if 'java' in x.lower() else 0)
df.java.value_counts()

0    745
1     36
Name: java, dtype: int64

In [16]:
# Signal processing
df['dsp'] = df['Job Description'].apply(lambda x: 1 if 'signal processing' in x.lower() else 0)
df.dsp.value_counts()

0    729
1     52
Name: dsp, dtype: int64

In [17]:
# Power
df['power'] = df['Job Description'].apply(lambda x: 1 if 'power' in x.lower() else 0)
df.power.value_counts()

0    394
1    387
Name: power, dtype: int64

In [18]:
# Machine learning
df['ML'] = df['Job Description'].apply(lambda x: 1 if 'machine learning' in x.lower() or 'deep learning' in x.lower()
                                       or 'neural network' in x.lower() or 'rnn' in x.lower() 
                                       or 'cnn' in x.lower() or 'lstm' in x.lower() or 'artificial intelligence' in x.lower() else 0)
df.ML.value_counts()

0    765
1     16
Name: ML, dtype: int64

In [19]:
# Embedded System
df['Embedded_System'] = df['Job Description'].apply(lambda x: 1 if 'embedded' in x.lower() else 0)
df.Embedded_System.value_counts()

0    684
1     97
Name: Embedded_System, dtype: int64

In [20]:
# Job description length
df['desc_len'] = df['Job Description'].apply(lambda x: len(x))
df[['desc_len']]

,desc_len
0,2209
1,4725
2,1852
3,3362
4,6141
...,...
951,3621
952,3818
953,7680
954,5141


## Parsing job seniority from job title 

In [21]:
# Position Senority
def seniority(title):
    if 'sr' in title.lower() or 'sr.' in title.lower() or 'manager'in title.lower() or 'senior' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
        return 'senior'
    elif 'jr' in title.lower() or 'jr.' in title.lower() or 'junior' in title.lower():
        return 'junior'
    elif 'entry' in title.lower() or 'associate' in title.lower():
        return 'entry'
    else:
        return 'na'
df['seniority'] = df['Job Title'].apply(seniority)
df.seniority.value_counts()

na        486
senior    278
junior     10
entry       7
Name: seniority, dtype: int64

## Store and read CSV 

In [22]:
df.to_csv('salary_data_cleaned.csv', index = False)

In [23]:
# Original
df = pd.read_csv('glassdoor_jobs.csv')
df.shape

(958, 14)

In [24]:
# Cleaned
df1 = pd.read_csv('salary_data_cleaned.csv')
df1.shape

(781, 35)